In [1]:
from glob import glob as gg
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import cm as cm
from matplotlib import rc as rc
#
from netCDF4 import Dataset  
#
from datetime import datetime,date
#
import pandas as pd

In [2]:
# mesh type
mesh_type='farc'

In [3]:
# spin-up
spinup_duration, spinup_unit = 2, 'm'

In [4]:
path_input='/home/fbirrien/NuArctic/nuarctic/REcoM1D/grid/'

path_input_mesh=path_input + '/mesh/' + mesh_type + '/'
path_input_track=path_input + 'track/'


In [5]:
#
# load vertical mesh
#
class vertical_mesh(object):
    def __init__(self, filename):
        self.read_vertical_mesh(filename)
    def read_vertical_mesh(self,filename):
        data = np.asarray(pd.read_csv(filename,header=None))
        self.nl = int(data[0])
        self.zbar = np.reshape(data[1:self.nl+1], len(data[1:self.nl+1]))
        self.Z = 0.5*(self.zbar[1:] + self.zbar[:-1])
 #       
filename=path_input_mesh + 'aux3d.out'
vmesh = vertical_mesh(filename)

In [6]:
#
# load MOSAiC track
#
class MOSAiC_track(object):
    def __init__(self,filename):
        self.read_MOSAiC_track(filename)
    def read_MOSAiC_track(self,filename):
        # open nc file
        ncid = Dataset(filename, "r", format="NETCDF4")        

        # read dates and geo coordinates
        self.dates = ncid.variables['dates'][:]
        self.longitude, self.latitude = ncid.variables['longitude'][:], ncid.variables['latitude'][:]
        if 'depth' in ncid.variables:
            self.depth = ncid.variables['depth'][:]
        # close nc file
        ncid.close()
#
#filename = path_input_track + 'Polarstern_trajectory.nc'
filename = path_input_track + 'MOSAiC_Icetrack_trajectory.nc'
track=MOSAiC_track(filename)
print ('track dates:')
print('start date', datetime.fromordinal(int(np.min(track.dates))))
print('end date', datetime.fromordinal(int(np.max(track.dates))))

track dates:
start date 2019-10-05 00:00:00
end date 2020-07-29 00:00:00


In [7]:
#
# load mesh + depth (truncated in the high latitude)
#
class Mesh(object):
    def __init__(self,path,latmin):
        self.read_mesh(path)
        self.read_depth(path)
        self.truncate_mesh(latmin)
    def read_mesh(self,path):
        filename=path+'nod2d.out'
        data = pd.read_csv(filename, sep='\s+', skiprows=1 , header=None)
        self.longitude, self.latitude = data.iloc[:,1], data.iloc[:,2]
    def read_depth(self,path):
        filename=path+'aux3d.out'
        data = np.asarray(pd.read_csv(filename,header=None))
        nb = int(data[0])
        self.depth = data[nb+1:]
    def truncate_mesh(self,latmin):
        indices=np.where(self.latitude>=latmin-0.5)[0]
        self.longitude, self.latitude, self.depth = self.longitude[indices], self.latitude[indices], self.depth[indices]
                 
mesh=Mesh(path_input_mesh, np.min(track.latitude))

In [8]:
#
# estimate depth along the MOSAiC trajectory
#
def tunnel_distance_nearest_neighbor_index(lon, lat, longitude_ref, latitude_ref):
    #
    # estimate nearest neighbor of (lon,lat) point and return point index
    #
    # convert lon, lat to radian
    deg2rad = np.pi / 180.0
    lon, lat, longitude_ref, latitude_ref =  deg2rad*lon, deg2rad*lat , deg2rad*longitude_ref, deg2rad*latitude_ref
    
    #compute cos,sin and cross-products for of the different a lon,lat arrays
    # grid
    clat, clon, slat, slon = np.cos(lat), np.cos(lon), np.sin(lat), np.sin(lon)
    clat_clon,  clat_slon = clat * clon, clat * slon
    
    # reference node
    clat_ref, clon_ref, slat_ref, slon_ref = np.cos(latitude_ref), np.cos(longitude_ref), np.sin(latitude_ref), np.sin(longitude_ref)
    clat_clon_ref,  clat_slon_ref = clat_ref * clon_ref, clat_ref * slon_ref
    
    # compute tunnel distance to the different grid nodes
    distance = []
    dX, dY, dZ = clat_clon - clat_clon_ref, clat_slon - clat_slon_ref, slat - slat_ref
    distance = dX**2 + dY**2 + dZ**2

    return np.argmin(distance)
# 
class Complete_track_data(object):
    def __init__(self, track, mesh):
        #data
        self.dates, self.longitude, self.latitude = track.dates, track.longitude, track.latitude
        # estimate depth
        self.estimate_depth(mesh)
        
    def estimate_depth(self,mesh):
        depth=[]
        for i in range(len(self.dates)):
            lon, lat = self.longitude[i],self.latitude[i]
            index=tunnel_distance_nearest_neighbor_index(mesh.longitude, mesh.latitude, lon, lat)
            depth.append(mesh.depth[index])
        self.depth=np.asarray(depth)
        
new_track=Complete_track_data(track,mesh)

In [9]:
#
# include nb of levels in use at each track point 
#
class vertical_track(object):
    def __init__(self,track, vmesh):
        self.dates, self.longitude, self.latitude, self.depth = track.dates, track.longitude, track.latitude, track.depth
        self.estimate_nb_of_level(vmesh)
    def estimate_nb_of_level(self,vmesh):
        # compare depth
        nlevels=[]
        for dpth in self.depth:
            nlevels.append(np.where(vmesh.zbar - dpth <0)[0][0])
        self.nlevels = np.asarray(nlevels)
#
track_data = vertical_track(new_track,vmesh)

In [10]:
#
# routine for time considerations
#
def leap_year(year):
    # define whether the selected year is a leap year
    flag= True if (((year%4==0) & (year%100!=0)) | (year%400==0)) else False
    return flag

def define_days_in_month(flag_leap_year):
    # define days in month according to selected years
    days_in_month = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31] if flag_leap_year else [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    return days_in_month

def convert_calendar_day_to_date(day_start,year_start):
    # convert FESOM start time input into calendar date (day,month,year)
    # check if the year is the leap year
    flag_leap_year=leap_year(year_start)
    
    # number of days per month
    days_in_month=define_days_in_month(flag_leap_year)
    days_sum = np.cumsum(days_in_month)
    
    # find dates of the start of the simulation
    diff = days_sum-day_start
    ind = np.where(diff>0)[0][0]
    day, month = days_in_month[ind]-diff[ind], ind+1

    return day, month, year_start

def estimate_time_length(length, length_unit, day, month, year):
    # estimate length of simulation/spinup in days according to input
    
    if 'd' in length_unit:
        length_day = length
    elif 'm' in length_unit:
        # estimate corresponding month and year
        monthu = month + length
        monthe = monthu%12 if monthu<12 else 12
        ny = int((monthu-monthu%12)/12)+1
        yeare=[year+i for i in range(ny)][-1]
        date_str1, date_str2 = str(year*10000+month*100+day), str(yeare*10000+monthe*100+day)
        length_day = datetime.strptime(date_str2,'%Y%m%d').toordinal()-datetime.strptime(date_str1,'%Y%m%d').toordinal()
    elif 'y' in length_unit:
        # corresponding years
        ny=length+1
        yeare = [year + i for i in range(ny)][-1]
        date_str1, date_str2 = str(year*10000+month*100+day), str(yeare*10000+month*100+day)
        length_day = datetime.strptime(date_str2,'%Y%m%d').toordinal()-datetime.strptime(date_str1,'%Y%m%d').toordinal()

    return length_day 

In [11]:

#
# estimate time axis of the simulation
#


def read_model_setup_parameters(filename):
    # read model setup parameters (namelist.config)
    
    with open(filename) as f:
        lines=f.readlines()
    for i,ll in enumerate(lines):
        if 'step_per_day' in ll:
            step_per_day=np.float(''.join(filter(lambda i: i.isdigit(), ll)))
        if 'run_length=' in ll:      
            run_length=int(''.join(filter(lambda i: i.isdigit(), ll)))
        if 'run_length_unit' in ll:
            run_length_unit =ll.split('=')[-1]
        if 'daynew' in ll:
            day_start=int(''.join(filter(lambda i: i.isdigit(), ll)))
        if 'yearnew' in ll:
            year_start=int(''.join(filter(lambda i: i.isdigit(), ll)))
    
    # time step (h)
    dt = step_per_day/24.
    
    # estimate day, month, year from the starting days (namelist.config)
    day, month, year = convert_calendar_day_to_date(day_start,year_start)
    
    # compute run_length in days
    run_length_day = estimate_time_length(run_length, run_length_unit, day, month, year)
        
    # create axis
    date_str=str(year*10000+month*100+day)
    date_start=datetime.strptime(date_str, '%Y%m%d').toordinal()
    date_end=date_start + run_length_day
    
    nt = int(run_length_day*(dt*24))
    dates=np.linspace(date_start, date_end, nt+1)
    
    return dates

#
path_config = '/home/fbirrien/NuArctic/nuarctic/REcoM1D/src/config/'
filename = path_config + 'namelist.config'
dates = read_model_setup_parameters(filename)
print ('time frame for the simulation (start & end dates)')
print (datetime.fromordinal(int(np.min(dates))),datetime.fromordinal(int(np.max(dates))))

time frame for the simulation (start & end dates)
2019-08-08 00:00:00 2020-06-03 00:00:00


In [12]:
#
# estimate indices and dates of spin-up
#
# start of the simulation
date_start=datetime.fromordinal(int(np.min(dates)))
day, month, year = date_start.day, date_start.month, date_start.year

# estimatre the duration of spin up in days
spinup_day_length = estimate_time_length(spinup_duration, spinup_unit, day, month, year)
date_end_spinup = np.min(dates) + spinup_day_length
print ('time frame for the spinup (start & end dates)')
print (date_start, datetime.fromordinal(int(date_end_spinup)))

time frame for the spinup (start & end dates)
2019-08-08 00:00:00 2019-10-08 00:00:00


In [13]:
#
# estimate coordinate, depth and level at each simulation time using MOSAiC track observations and manage spinup
#
class simulation_data(object):
    def __init__(self, dates, vmesh, track, spinup):
        self.dates = dates
        self.nl, self.zbar, self.Z = vmesh.nl, vmesh.zbar, vmesh.Z
        # estimate coordinates, depth and 
        self.corresponding_track_data(track)
        # introduce simulation spinup
        self.estimate_simulation_spinup(spinup)
        
    def corresponding_track_data(self, track):
        #initialization
        thr = np.sum(np.diff(track.dates))/(len(track.dates)-1)
        lon, lat, dpth, nlvl = [], [], [], []

        for dt in dates:
            
            # look for corresponding dates
            index = np.argmin(abs(track.dates-dt))
            if abs(track.dates[index]-dt)<thr:
                #check for corresponding data within a certain interval period
                lon.append(track.longitude[index]), lat.append(track.latitude[index])
                dpth.append(track.depth[index]), nlvl.append(track.nlevels[index])
            else:
                lon.append(np.nan), lat.append(np.nan)
                dpth.append(np.nan), nlvl.append(np.nan)
                if (track.dates[index]-dt)<0:
                    print ('actual date: ', datetime.fromordinal(int(dt)))
                    print ('date at the end of the track', datetime.fromordinal(int(np.max(track.dates))))
                    print('warning: check correspondance of time between end of the simulation and end of the track.')
                else:
                    print ('actual date: ', datetime.fromordinal(int(dt)))
                    print ('date at the start of the track', datetime.fromordinal(int(np.min(track.dates))))
                    print('warning: check correspondance of time between start of the simulation and start of the track. only the spinup can compensate a NaN gap.')
        # store data
        self.longitude, self.latitude, self.depth, self.nlevels = np.asarray(lon), np.asarray(lat), np.asarray(dpth), np.asarray(nlvl)
    
    def estimate_simulation_spinup(self,spinup):
        # determine coordinates, depth and level for the spinup (=conditions at the end of spinup)
        index=np.argmin(abs(self.dates-spinup))
        lon_ref, lat_ref, depth_ref, nlvl_ref = self.longitude[index],self.latitude[index], self.depth[index], self.nlevels[index]
        
        # fill spinup time with reference values
        self.longitude[:index], self.latitude[:index] = lon_ref, lat_ref
        self.depth[:index], self.nlevels[:index] = depth_ref, nlvl_ref
#
simu_data = simulation_data(dates,vmesh, track_data, date_end_spinup)

actual date:  2019-08-08 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-08-08 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-08-08 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-08-08 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-08-08 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-08-08 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-08-08 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-08-08 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-08-08 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-08-08 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-08-08 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-08-08 00:00:00
date at the start of the track 

actual date:  2019-08-24 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-08-24 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-08-24 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-08-24 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-08-24 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-08-24 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-08-24 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-08-24 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-08-24 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-08-24 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-08-24 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-08-24 00:00:00
date at the start of the track 

date at the start of the track 2019-10-05 00:00:00
actual date:  2019-09-01 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-09-01 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-09-01 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-09-01 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-09-01 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-09-01 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-09-01 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-09-01 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-09-01 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-09-01 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-09-01 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  

date at the start of the track 2019-10-05 00:00:00
actual date:  2019-09-15 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-09-15 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-09-15 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-09-15 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-09-15 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-09-15 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-09-15 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-09-15 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-09-15 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-09-15 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  2019-09-16 00:00:00
date at the start of the track 2019-10-05 00:00:00
actual date:  

/usr/lib/python3/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [14]:
#
# save data to netcdf file
#
class Output(object):
    def __init__(self, filename, data):
        self.write_grid_information(filename,data)
        
    def write_grid_information(self,filename,data):
        # store grid (Lagrangian and vertical) information
        ncid = Dataset(filename, "w", format="NETCDF4") 

        ## define dimensionss
        ncid.createDimension('time', len(data.dates))
        ncid.createDimension('nl', data.nl)
        ncid.createDimension('nl1', data.nl-1)
        dimnl, dimnl1, dimt = ('nl'), ('nl1'), ('time')

        ## create variables
        # dates
        dt = ncid.createVariable('dates', "f8",'time')
        
        # geo coordinates
        lon, lat = ncid.createVariable('longitude', "f8",'time'), ncid.createVariable('latitude', "f8",'time')
        
        # vertical mesh
        zb, z = ncid.createVariable('zbar', "f8",'nl'), ncid.createVariable('Z', "f8",'nl1')
        
        # related depth and number of vertical levels along the track
        dpth, nlvl = ncid.createVariable('depth', "f8",'time'), ncid.createVariable('nlevels', "f8",'time')
        
        
        ## variables attributes
        # description
        dt.description='dates related to python ordinal dates (reference date 01/01/0001)'
        lon.description, lat.description = 'longitude coordinates', 'latitude coordinates'
        zb.description, z.description = 'vertical discretization of depth axis (from ' + mesh_type + ' mesh)', 'vertical discretization of depth axis (middle of cells)'
        dpth.description, nlvl.description = 'depth (from topography)', 'number of level considered in simulation (according to depth)'
        # units
        dt.units='days'
        lon.units, lat.units = 'degree (east)', 'degree (north)'
        zb.units, z.units = 'm', 'm'
        dpth.units, nlvl.units = 'm', ''        
        
        ## fill variables
        # dates
        ncid['dates'][:] = data.dates
        
        # geo coordinates
        ncid['longitude'][:] , ncid['latitude'][:] =  data.longitude, data.latitude
        
        # vertical mesh
        print(len(data.zbar), len(data.Z))
        ncid['zbar'][:] , ncid['Z'][:] = data.zbar, data.Z
        
        # depth and vertical levels along the track
        ncid['depth'][:] , ncid['nlevels'][:] = data.depth, data.nlevels
        
        #
        ncid.close()

#
filename = path_input + 'REcoM1D_mesh.nc'
Output(filename, simu_data)

48 47
